In [1]:
%load_ext autoreload

In [2]:
%autoreload now

In [3]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-08137aa2-e69b-5e74-8390-7997329b1336"
os.environ["WANDB_API_KEY"] = ""

# Download and convert data

In [4]:
from document_segmentation.pagexml.annotations.renate_analysis import (
    RenateAnalysis,
    RenateAnalysisInv,
)
from document_segmentation.pagexml.annotations.sheet import Sheet
from document_segmentation.settings import RENATE_ANALYSIS_DIR, RENATE_ANALYSIS_SHEETS

sheets: list[Sheet] = [RenateAnalysis()] + [
    RenateAnalysisInv(sheet) for sheet in RENATE_ANALYSIS_SHEETS
]

In [5]:
N = None

for sheet in sheets:
    sheet.download(RENATE_ANALYSIS_DIR, N)

Writing documents: 0doc [00:00, ?doc/s]
Writing documents: 100%|██████████| 26/26 [00:11<00:00,  2.36doc/s]
Writing documents: 184doc [00:25,  7.32doc/s]                      


# Load Data

In [6]:
TRAINING_DATA = 0.8

In [7]:
from document_segmentation.model.dataset import DocumentDataset

dataset: DocumentDataset = DocumentDataset.from_dir(RENATE_ANALYSIS_DIR)
dataset.shuffle()

len(dataset)

Reading JSON files: 100%|██████████| 285/285 [00:01<00:00, 183.94file/s]


285

In [8]:
dataset._class_counts()

Counter({<Label.IN: 1>: 3206,
         <Label.BEGIN: 0>: 285,
         <Label.END: 2>: 281,
         <Label.OUT: 3>: 170})

In [9]:
dataset.class_weights()

[0.9965034965034965,
 0.08886810102899906,
 1.0106382978723405,
 1.6666666666666667]

In [10]:
training_data, test_data = dataset.split(TRAINING_DATA)

In [11]:
training_data._class_counts()

Counter({<Label.IN: 1>: 2648,
         <Label.BEGIN: 0>: 228,
         <Label.END: 2>: 226,
         <Label.OUT: 3>: 113})

In [12]:
test_data._class_counts()

Counter({<Label.IN: 1>: 558,
         <Label.BEGIN: 0>: 57,
         <Label.OUT: 3>: 57,
         <Label.END: 2>: 55})

# Train Model

In [13]:
BATCH_SIZE = 128
EPOCHS = 10
WEIGHTS = dataset.class_weights()

In [52]:
%autoreload now

In [53]:
from document_segmentation.model.page_sequence_tagger import PageSequenceTagger

tagger = PageSequenceTagger()

In [54]:
tagger._device

'mps'

In [55]:
tagger

PageSequenceTagger(
  (_page_embedding): PageEmbedding(
    (_region_model): RegionEmbeddingSentenceTransformer(
      (_transformer_model): SentenceTransformer(
        (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: RobertaModel 
        (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
      )
      (_region_type): Embedding(9, 16)
      (_linear): Linear(in_features=784, out_features=512, bias=True)
    )
    (_rnn): LSTM(512, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
    (_linear): Linear(in_features=512, out_features=256, bias=True)
  )
  (_rnn): LSTM(256, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (_linear): Linear(in_features=512, out_feature

In [56]:
tagger.train_(training_data, EPOCHS, BATCH_SIZE, WEIGHTS)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Evaluating: 100%|██████████| 229/229 [00:03<00:00, 72.20batch/s]
        [1],
        [2]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.
Evaluating: 100%|██████████| 229/229 [00:03<00:00, 72.24batch/s] 
        [1],
        [2]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.
Evaluating: 100%|██████████| 229/229 [00:03<00:00, 71.52batch/s]
        [1],
        [2]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.
Evaluating: 100%|██████████| 229/229 [00:03<00:00, 66.86batch/s]
        [1],
        [2]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.
Evaluating: 100%|██████████| 229/229 [00:03<00:00, 69.11batch/s]
        [1],
        [2]]) classes have zero instances in both the predictions and the ground truth la

MulticlassAccuracy,▁▁▁▁▁▁▁▁▁▁
batch_size,▁▁▁▂▁▁▁▁▁▁▃▄▄▁▂▃█▁▄▁▂▁▁▁▅▁▃▃▁▁▁▁▁▁▁▂▂▁▁▁
loss,███▄▃█▆▄██▁▁▁▃▂▂▁▄▁█▂▆▆█▁█▂▁█▄█▆█▆▃▂▂██▄
MulticlassAccuracy,0.03515
batch_size,2
loss,1.74989


In [57]:
import torch

with open("page_sequence_tagger.pt", "xb") as f:
    torch.save(tagger, f)

# Evaluate Model

In [58]:
import sys

precision, recall, f1, accuracy = tagger.eval_(test_data, BATCH_SIZE, sys.stdout)

Predicted	Actual	Page ID	Text	Scores


Evaluating:   0%|          | 0/57 [00:00<?, ?batch/s]

OUT	BEGIN	NL-HaNA_1.04.02_1557_0257.jpg	Van Palembang onder dato 18=en September 1694.; De	[5.6757439637067364e-08, 9.821837920753751e-08, 9.368542208676445e-08, 0.9999997615814209]
OUT	IN	NL-HaNA_1.04.02_1557_0258.jpg	Van Palembang onder dato 18=en Septemb: 1694.; bei	[1.73753420540379e-08, 2.0216736729139484e-08, 1.2059354403959333e-08, 1.0]
OUT	END	NL-HaNA_1.04.02_1557_0259.jpg	Van Palembang onder dato 18=en Septemb: 1694; zede	[1.094604371587593e-07, 8.344021296124993e-08, 3.735721776365608e-08, 0.9999997615814209]


Evaluating:   4%|▎         | 2/57 [00:00<00:07,  7.44batch/s]

OUT	BEGIN	NL-HaNA_1.04.02_1480_0222.jpg	door voortdraght; 258: 6: vermiljoen, â 108 roa dm	[5.627558863352533e-08, 9.988809779315488e-08, 1.0020409746402947e-07, 0.9999997615814209]
OUT	IN	NL-HaNA_1.04.02_1480_0223.jpg	de man volgens gebruijk; gerekent; wolffdas kort n	[1.7119246464858406e-08, 2.0450364957014244e-08, 1.2638238899853604e-08, 1.0]
OUT	END	NL-HaNA_1.04.02_1480_0224.jpg	moeste hij daar omme, andermaal, niet te huijs bli	[1.1012573963853356e-07, 8.463030098937452e-08, 3.7651791018333824e-08, 0.9999997615814209]


...

OUT	BEGIN	NL-HaNA_1.04.02_1147_0807.jpg	Inden Name Godes Amen, Cout ende; kennelijck zij e	[5.663880742190486e-08, 9.833441083628713e-08, 9.427570546449715e-08, 0.9999997615814209]
OUT	IN	NL-HaNA_1.04.02_1147_0808.jpg	metten anderen tot zijnder gedachtenisse sullen; v	[1.732576748736392e-08, 2.026460776960448e-08, 1.212920341941981e-08, 1.0]
OUT	END	NL-HaNA_1.04.02_1147_0809.jpg	d' h:ren weesmeesteren, als oock andere magistrate	[1.0927457338993918e-07, 8.351248936833144e-08, 3.7460250013054974e-08, 0.9999997615814209]


Evaluating:  96%|█████████▋| 55/57 [00:19<00:00,  5.55batch/s]

OUT	BEGIN	NL-HaNA_1.04.02_1557_0221.jpg	Van Palembang onder dato 28=en Iulij A:o 1694.; mo	[5.6020706296067146e-08, 9.86056178930994e-08, 1.206326771807653e-07, 0.9999997615814209]
OUT	IN	NL-HaNA_1.04.02_1557_0222.jpg	Van Palembang onder dato 28=en Iulij ao 1698.; ove	[1.1020891754753848e-08, 1.6889622145299654e-08, 1.181945297190623e-08, 1.0]
OUT	IN	NL-HaNA_1.04.02_1557_0223.jpg	Van Palembang onder dato 28=en Iulij A=o 1694.; re	[8.888533464812554e-09, 1.154821038795717e-08, 7.28675519923172e-09, 1.0]
OUT	IN	NL-HaNA_1.04.02_1557_0224.jpg	Van Palembang onder dato 28=en Julij A=o 1694.; ge	[9.675612311355053e-09, 1.1001801247800813e-08, 6.311431377525878e-09, 1.0]
OUT	IN	NL-HaNA_1.04.02_1557_0225.jpg	Van Palembang den 28=en; Julij A=o 1694; versoek o	[1.0093375024666784e-08, 1.0528792948605314e-08, 6.250811868113715e-09, 1.0]
OUT	IN	NL-HaNA_1.04.02_1557_0226.jpg	Van Palembang onder dato 28=en Iulij A:o 1699.; de	[1.4493211963895192e-08, 1.4006209525518898e-08, 7.08600289556216e-09, 1.0]

Evaluating: 100%|██████████| 57/57 [00:27<00:00,  2.06batch/s]

OUT	OUT	NL-HaNA_1.04.02_1547_0489.jpg		[3.6810865822189953e-07, 8.309551162710704e-07, 1.1451136288087582e-06, 0.999997615814209]
OUT	OUT	NL-HaNA_1.04.02_1547_0490.jpg		[3.451520314001755e-08, 5.1552522251085975e-08, 4.397015729296072e-08, 0.9999998807907104]
OUT	OUT	NL-HaNA_1.04.02_1547_0491.jpg	Generale Monster Rolle van Alle; des EComp. s zoon	[2.166946266868308e-08, 2.6385757578850644e-08, 1.7646607730625874e-08, 1.0]
OUT	OUT	NL-HaNA_1.04.02_1547_0492.jpg		[2.0537376244078587e-08, 2.3109695490575177e-08, 1.4110515422771641e-08, 1.0]
OUT	OUT	NL-HaNA_1.04.02_1547_0493.jpg		[2.044451363758526e-08, 2.2344833539023057e-08, 1.3156687295179381e-08, 1.0]
OUT	OUT	NL-HaNA_1.04.02_1547_0494.jpg		[2.0476749185149856e-08, 2.2118527454040304e-08, 1.2821038453125766e-08, 1.0]
OUT	BEGIN	NL-HaNA_1.04.02_1547_0495.jpg		[2.048331282367144e-08, 2.2013182388036512e-08, 1.266985627523809e-08, 1.0]
OUT	IN	NL-HaNA_1.04.02_1547_0496.jpg	1729„ Persoonen soo van de Commersie Militie als a	[2.0028060987442586

In [61]:
import csv
import sys

from document_segmentation.pagexml.datamodel.label import Label

writer = csv.DictWriter(
    sys.stdout, fieldnames=["Metric"] + [label.name for label in Label], delimiter="\t"
)

writer.writeheader()

for metric in precision, recall, f1:
    assert metric.average is None

    scores: list[float] = metric.compute().tolist()
    writer.writerow(
        {"Metric": metric.__class__.__name__}
        | {label.name: f"{score:.4f}" for label, score in zip(Label, scores)}
    )

assert accuracy.average is not None

print(
    f"{accuracy.__class__.__name__} ({accuracy.average} average):\t{accuracy.compute().item():.4f}",
    file=sys.stdout,
)

        [1],
        [2]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.


Metric	BEGIN	IN	END	OUT
MulticlassPrecision	0.0000	0.0000	0.0000	0.0784
MulticlassRecall	0.0000	0.0000	0.0000	1.0000
MulticlassF1Score	0.0000	0.0000	0.0000	0.1454
MulticlassAccuracy (micro average):	0.0784
